# 数据分析

在本节中，我们将介绍**空间数据分析**（Spatial Data Analysis），这一领域通常也被称为
[空间分析（Spatial Analysis）](https://en.wikipedia.org/wiki/Spatial_analysis)，
有时也被称为空间统计（Spatial Statistics）。

> 空间分析或空间统计，指的是一切利用对象的**拓扑属性、几何属性或地理属性**来研究实体的正式技术方法。  
> 引自 [Wikipedia](https://en.wikipedia.org/wiki/Spatial_analysis)。

维基百科实际上提供了一个[非常不错的概览](https://en.wikipedia.org/wiki/Spatial_analysis)，
可以帮助你快速理解空间分析主要研究的内容以及它所涵盖的范围。

## 使用 Fiona 与 Shapely 进行分析

让我们来探索一些 Shapely 中用于 **（拓扑）空间关系（Topological Spatial Relationships）** 的函数。
下面给出了一个简化示意图，更多内容可参见
[Wikipedia](https://en.wikipedia.org/wiki/Spatial_relation)。

![(Topological) Spatial Relationships](images/spatialrelations.png)
*拓扑空间关系示例 —— [作者 Krauss，自制作品，CC BY-SA 3.0](https://commons.wikimedia.org/w/index.php?curid=21299138)*

如果你希望深入了解其理论背景，可以参考
[九交模型（Dimensionally Extended nine-Intersection Model, DE-9IM）](https://en.wikipedia.org/wiki/DE-9IM)。

我们将从一个简单的问题开始：

**巴拉那河（Paraná）是否流经阿根廷？**

我们将使用两套 Natural Earth 数据集：
- 河流与湖泊（Rivers and Lakes，几何类型为 LineString）
- 国家边界（Countries，Admin-0，多边形）

用空间关系的术语来表述，这个问题等价于：

> *表示巴拉那河的（Multi）LineString 是否穿过（crosses）表示阿根廷的（Multi）Polygon？*

我们将按照以下步骤进行：

- 读取河流与湖泊数据集
- 提取巴拉那河要素的几何对象  
- 读取国家数据集
- 提取阿根廷要素的几何对象  
- 使用 Shapely 的 `crosses` 函数进行判断
- （进阶）列出这两个几何对象之间所有的 DE-9IM 关系
- （进阶）巴拉那河流经了哪些国家？

In [5]:
import fiona
from shapely.geometry import shape

### 获取巴拉那河（Paraná）的几何对象

In [6]:
with fiona.open("../data/rivers_lake_centerlines.gpkg") as rivers_lakes:
    for feature in rivers_lakes:
        if feature['properties']['name'] == 'Paraná':
            parana_geom = shape(feature["geometry"])
            print(f'Found Paraná! geom type: {parana_geom.geom_type}')
            break

Found Paraná! geom type: LineString


### 获取阿根廷（Argentina）的几何对象

In [7]:
with fiona.open("../data/countries.json") as countries:
    for feature in countries:
        if feature["properties"]["NAME"] == "Argentina":
            argentina_geom = shape(feature["geometry"])
            print(f'Found Argentina! geom type: {argentina_geom.geom_type}')
            break


Found Argentina! geom type: MultiPolygon


### 巴拉那河（Paraná）是否流经阿根廷？


In [8]:
parana_geom.crosses(argentina_geom)


True

### 列出（DE-9IM）空间关系

In [9]:
parana_geom.relate(argentina_geom)

'101FF0212'

### 巴拉那河（Paraná）流经哪些国家？


In [10]:
print("The Paranà floats through:")

with fiona.open("../data/countries.gpkg") as countries:
    for feature in countries:
        country_geom = shape(feature["geometry"])
        if parana_geom.crosses(country_geom):
            print(feature["properties"]["NAME"])


The Paranà floats through:
Argentina
Brazil
Paraguay


# 大气总臭氧数据分析

[世界臭氧与紫外辐射数据中心（World Ozone and Ultraviolet Radiation Data Centre，WOUDC）](https://woudc.org)
是世界气象组织（World Meteorological Organization，WMO）全球大气观测计划
（Global Atmosphere Watch, GAW）下属的六个世界数据中心之一。

WOUDC 提供了**每日大气总臭氧**的数据档案，
该数值表示从地表到大气边缘这一整根大气柱中的臭氧总量。
这些数据来自全球范围内分布的监测站网络。

大气总臭氧的计量单位是
[多布森单位（Dobson units, DU）](https://en.wikipedia.org/wiki/Dobson_unit)，
其合理范围通常为 **100–700 DU**。

下面我们将基于下载得到的 **GeoJSON 数据**
（通过 [WFS 服务](https://geo.woudc.org/ows?service=WFS&version=1.1.0&request=GetFeature&outputformat=GeoJSON&typename=totalozone&filter=%3Cogc:Filter%3E%3Cogc:And%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Eplatform_id%3C/PropertyName%3E%3CLiteral%3E493%3C/Literal%3E%3C/PropertyIsEqualTo%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Eplatform_type%3C/PropertyName%3E%3CLiteral%3ESTN%3C/Literal%3E%3C/PropertyIsEqualTo%3E%3Cogc:PropertyIsBetween%3E%3Cogc:PropertyName%3Einstance_datetime%3C/ogc:PropertyName%3E%3Cogc:LowerBoundary%3E1924-01-01%2000:00:00%3C/ogc:LowerBoundary%3E%3Cogc:UpperBoundary%3E2021-12-31%2023:59:59%3C/ogc:UpperBoundary%3E%3C/ogc:PropertyIsBetween%3E%3C/ogc:And%3E%3C/ogc:Filter%3E&sortby=instance_datetime%20DESC&startindex=0&maxfeatures=200000)），
生成一条时间序列图，并对数据做一些基本的取值范围检查。

所使用的数据来自
[CNRS 里奥加耶戈斯（Rio Gallegos）监测站（站点编号 493）](https://woudc.org/data/stations/?id=493&lang=en)，
涵盖其完整的历史观测记录。


In [18]:
from datetime import datetime
import json
import gzip

with gzip.open('../data/totalozone-station-493.json.gz') as gzfh:
        data = json.load(gzfh)
 
len(data['features'])

5521

In [19]:
# setup graph axes
x_axis = [datetime.strptime(x['properties']['instance_datetime'], '%Y/%m/%d 00:00:00+00') for x in data['features']]
y_axis = [float(x['properties']['daily_columno3']) for x in data['features']]

# average
total_average = sum(y_axis) / float(len(y_axis))
total_average

304.82177141822444

In [20]:
# render simple plot

from bokeh.plotting import figure, output_notebook, show

# output graph to notebook
output_notebook()

p = figure(title='Bucharest 226 dobson', x_axis_type='datetime', y_axis_label='Dobson units')
p.line(x_axis, y_axis, legend_label="Dobson units", line_width=2)
show(p)

Loading BokehJS ...

让我们检查一下，是否存在任何数值落在可接受范围之外：

In [21]:
max(y_axis) > 700 or min(y_axis) < 100

False

现在我们尝试将所有 [WOUDC 监测站](https://woudc.org/data/stations/)
导入到 [GeoPandas](https://geopandas.org) 中，并开展进一步的分析。

首先，将监测站以表格形式展示，并按 **WMO 区域** 进行分组：

In [22]:
import geopandas

gdf = geopandas.read_file('../data/woudc-stations.geojson', engine='fiona').to_crs('epsg:3857')

gdf.groupby('wmo_region').wmo_region.count()

wmo_region
I                 34
II                56
III               37
IV               153
V                 32
VI               191
the Antarctic     34
Name: wmo_region, dtype: int64

现在我们将这些监测站绘制到地图上


In [23]:
from bokeh.models import ColumnDataSource

p = figure(tools='pan, wheel_zoom', x_axis_type='auto', y_axis_type='auto', width=800, height=500)

# We need to make x,y columns for Bokeh to display
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

gdf['x'] = gdf.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
gdf['y'] = gdf.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)
gdf = gdf.drop('geometry', axis=1).copy()

psource = ColumnDataSource(gdf)

# add populated places point overlay
p.scatter(x='x', y='y', size=10, alpha=0.7, source=psource, color='blue', legend_label='WMO WOUDC stations')
# add background tiles layer from CARTO
p.add_tile("CartoDB Positron", retina=True)

show(p)

接下来我们只绘制 **WMO 第 III 区域**（即南美洲）的监测站


In [24]:
gdf2 = gdf[gdf.wmo_region.isin(['III']) == True]

psource2 = ColumnDataSource(gdf2)
p.scatter(x='x', y='y', size=10, alpha=0.7, source=psource2, color='red', legend_label='WMO WOUDC stations in WMO Region III')
show(p)

---
[<- 栅格数据（Raster data）](05-raster-data.ipynb) | [Visualization ->](07-visualization.ipynb)
